In [ ]:
!pwd

In [1]:
import pprint
pp = pprint.PrettyPrinter(indent=4)

In [2]:
import pandas as pd

In [3]:
import datetime

In [4]:
import pymongo
from pymongo import MongoClient
# db_client = MongoClient('localhost', 27017)
db_client = MongoClient('172.19.31.5', 27017)
db = db_client['inspirehep']
articles = db['articles']        # use the "articles" collection of the database

In [5]:
import networkx as nx

In [6]:
from tqdm.notebook import tqdm

In [7]:
# logging library
import logging
# logging.basicConfig(filename='app.log', level=logging.DEBUG, format='%(funcName)s-\t\t\t\t%(message)s')

import requests
from requests.exceptions import HTTPError

import json, sys

In [8]:
BASE_URL = r'https://inspirehep.net/api/literature/'

def api_call(_id:int):

    try:
        response = requests.get(
            BASE_URL+str(_id),
            )

        # If the response was successful, no Exception will be raised
        response.raise_for_status()

    except HTTPError as http_err:
        logging.exception("HTTP exception occurred")
        if response.status_code == 429:             # when hit the rate limit
            timeout = response.headers['x-retry-in']
            logging.debug('rate limited, sleeping for {}s'.format(timeout))
            time.sleep(timeout)
            return api_call(params)                 # try again recursevely
        elif response.status_code == 410:
            
        else:                                       # another HTTP error
            sys.exit()

    except Exception as err:
        logging.exception("non-HTTP exception occurred")
        sys.exit()
    
    else:   # Success on API call
        json_text = response.text
        # remove $ from keys to avoid conflicts with database
        json_text = json_text.replace(r'$ref','ref')      
        json_text = json_text.replace(r'$schema','schema')
        # replace "id" label to "_id" to use as database id
        json_text = json_text.replace(r'"id"','"_id"')
        logging.debug('data succesfully obtained and parsed')

        return json.loads(json_text)

In [9]:
def insert_one_to_db(doc:dict):

    # insert a single document to DB if is not already in collection.
    logging.debug('inserting document id:{}'.format(doc['_id']))
    try:
        # insert document to database using doc id as database id
        db_result = articles.insert_one(doc)
        return True
    except pymongo.errors.DuplicateKeyError:
        # skip document because it already exists in collection
        logging.debug("duplicated key, document already in DB")
        return False
    except Exception as err:
        logging.exception("unknown error while inserting to db")

## Get documents from database

Get a pointer to all the documents in the articles collection

In [10]:
cursor = articles.find({},{'_id':1, 'citations_ids':1})

make a list of ids

In [11]:
ids = []
for article in cursor:
    ids += [article['_id']]

In [12]:
print(len(ids))

17150


Get a single document to see the schema (dict keys)

In [13]:
article = list(articles.find({'_id':ids[0]}))

In [14]:
pp.pprint(article[0].keys())

dict_keys(['links', 'metadata', 'created', 'updated', '_id', 'citations_ids'])


In [15]:
pp.pprint(article[0]['metadata'].keys())

dict_keys(['referenced_authors_bais', 'citation_count_without_self_citations', 'license', 'schema', 'curated', 'facet_author_name', 'control_number', '_oai', 'core', 'citation_count', 'acquisition_source', 'titles', 'texkeys', 'abstracts', 'document_type', 'earliest_date', 'author_count', 'number_of_pages', 'first_author', 'legacy_version', 'authors', 'figures', 'preprint_date', 'arxiv_eprints', 'citeable', 'inspire_categories', 'public_notes', 'references', 'legacy_creation_date', 'collaborations', 'report_numbers'])


In [16]:
article[0]['created']

'2020-11-04T00:00:00+00:00'

In [17]:
article[0]['updated']

'2020-11-04T15:44:01.959966+00:00'

In [18]:
article[0]['metadata']['titles']

[{'source': 'arXiv',
  'title': 'Study of $B^0_s \\rightarrow J/\\psi\\pi^+\\pi^-K^+K^-$ decays'}]

In [19]:
article[0]['metadata']['collaborations']

[{'value': 'LHCb'}]

get again documents with the interesting properties

In [20]:
cursor = articles.find(
    {},   # query search
    {'_id': 1, 
     'citations_ids': 1, 
     'created': 1,
     'metadata.titles.title': 1,
     'metadata.inspire_categories': 1,
     'metadata.document_type': 1,
     'metadata.publication_info.journal_title':1
    }    # fields contained in result
)

In [21]:
documents = list(cursor)

### Build network

create network

In [22]:
G = nx.Graph()

add nodes with metadata to the network

In [23]:
nodes = [(doc['_id'], 
          {'title': doc['metadata']['titles'][0]['title'],
           'year': datetime.datetime.strptime(doc['created'], "%Y-%m-%dT%H:%M:%S+00:00").year,
           'document_type': doc['metadata']['document_type'],
           'journal':[journal['journal_title'] for journal in doc['metadata']['publication_info'] if 'journal_title' in journal.keys()] if 'publication_info' in doc['metadata'].keys() else [],
           'categories': [category['term'] for category in doc['metadata']['inspire_categories']] if 'inspire_categories' in doc['metadata'].keys() else []
          }) 
         for doc in documents]

In [24]:
G.add_nodes_from(nodes)

build edges from data

In [31]:
def get_title_byid(doc_id):
    # if document already in DB
    for doc in documents:
        if doc['_id'] == doc_id:
            return doc['metadata']['titles'][0]['title']
        
    # else search and append to DB
    article_json = api_call(doc_id)
    insert_one_to_db(article_json)
    print(article_json['metadata']['titles'][0]['title'],end='\x1b[1K\r')
    
    return article_json['metadata']['titles'][0]['title']
    
edges = []
for doc in tqdm(documents):
    if 'citations_ids' in doc.keys():
        for cite_id in doc['citations_ids']:
            if cite_id == '1765645': 
                pass
            else:
#                 print(doc['_id'], doc['metadata']['titles'][0]['title'],end='\r')
                edge = (doc['_id'],cite_id,
                        {'title_cited_article': doc['metadata']['titles'][0]['title'],
                         'title_citing_article': get_title_byid(cite_id)})
                edges += [edge]

ERROR:root:HTTP exception occurred
Traceback (most recent call last):
  File "<ipython-input-8-391cd11f9682>", line 11, in api_call
    response.raise_for_status()
  File "/usr/local/lib/python3.5/dist-packages/requests-2.22.0-py3.5.egg/requests/models.py", line 940, in raise_for_status
    raise HTTPError(http_error_msg, response=self)
requests.exceptions.HTTPError: 410 Client Error: GONE for url: https://inspirehep.net/api/literature/1735651


SystemExit: 

In [ ]:
for doc in tqdm(documents):
    if 'citations_ids' in doc.keys():
        for cite_id in doc['citations_ids']:
            result = list(articles.find(
                {'_id': cite_id},   # query search
                {'_id': 1, 
                 'citations_ids': 1, 
                 'created': 1,
                 'metadata.titles.title': 1,
                 'metadata.inspire_categories': 1,
                 'metadata.document_type': 1,
                 'metadata.publication_info.journal_title':1
                }    # fields contained in result
            ))
            if len(result) == 0 or len(result) != 1:
                print(doc['_id'], cite_id, len(result))

1691854 1765645 0
1691634 1765645 0
1677076 1765645 0
1676179 1765645 0
1672475 1735651 0
1672475 1688783 0
1673136 1753024 0
1673136 1728505 0
1672368 1754902 0
1672883 1778518 0
1672883 1712701 0
1672883 1677167 0
1672883 1696542 0
1672884 1801557 0
1672884 1731800 0
1672884 1799248 0
1672147 1705313 0
1672147 1698408 0
1672147 1678752 0
1672011 1811554 0
1672011 1762446 0
1671777 1677568 0
1671777 1799246 0
1671777 1768720 0
1672099 1711831 0
1672099 1731081 0
1672099 1690861 0
1672099 1673419 0
1672099 1793923 0
1672099 1765601 0
1671792 1801632 0
1671792 1670785 0
1671792 1710007 0
1671792 1703040 0
1671792 1737401 0
1671792 1773987 0
1671792 1790422 0
1671792 1682786 0
1671810 1794160 0
1671810 1699539 0
1671810 1783170 0
1671810 1695180 0
1671810 1788928 0
1671810 1787885 0
1671499 1709843 0
1671499 1738830 0
1671499 1767700 0
1670803 1713251 0
1670803 1675042 0
1670803 1677702 0
1670803 1827792 0
1670020 1704661 0
1670020 1706111 0
1670020 1778653 0
1670020 1680882 0
1670168 16

1657384 1763730 0
1657384 1752034 0
1657341 1726885 0
1656995 1817829 0
1656995 1679397 0
1656995 1800022 0
1656834 1788928 0
1656834 1787885 0
1656834 1787966 0
1656834 1767872 0
1656834 1754080 0
1657017 1782760 0
1657017 1727098 0
1655968 1737053 0
1654926 1796819 0
1654926 1672145 0
1654926 1753396 0
1654814 1699539 0
1654814 1643035 0
1654814 1733262 0
1654814 1701431 0
1654814 1773526 0
1654814 1802504 0
1654582 1765710 0
1654582 1608471 0
1654582 1663572 0
1654582 1677681 0
1654582 1728246 0
1654582 1701785 0
1654582 1713817 0
1654582 1680852 0
1654582 1714944 0
1654582 1670653 0
1654582 1684504 0
1654582 1677911 0
1654582 1679353 0
1654582 1692365 0
1654582 1698463 0
1654582 1688842 0
1654582 1736767 0
1654582 1670949 0
1654582 1701199 0
1654582 1802383 0
1654582 1693823 0
1654582 1684641 0
1654582 1788075 0
1654582 1667581 0
1654582 1754536 0
1654582 1762048 0
1654582 1692605 0
1654582 1747465 0
1654582 1678318 0
1654582 1668850 0
1654582 1708729 0
1654582 1757292 0
1654582 16

1644362 1772421 0
1644362 1710309 0
1644362 1756141 0
1644362 1769799 0
1644362 1751235 0
1644362 1769801 0
1644362 1813325 0
1644362 1674988 0
1644362 1694561 0
1644362 1680726 0
1644362 1761385 0
1644362 1658059 0
1644362 1797328 0
1644362 1713142 0
1644362 1663384 0
1644362 1664565 0
1644363 1729424 0
1644363 1771755 0
1644363 1679185 0
1644363 1695387 0
1644363 1655924 0
1644363 1680882 0
1644363 1677916 0
1644363 1695047 0
1644363 1729763 0
1644363 1680680 0
1644363 1773588 0
1644363 1651803 0
1644363 1724679 0
1644363 1656594 0
1644363 1697550 0
1644363 1762701 0
1644099 1699539 0
1644364 1756076 0
1644364 1730162 0
1644364 1806002 0
1644364 1739964 0
1644364 1613684 0
1644364 1664415 0
1644364 1734153 0
1644364 1707945 0
1644364 1756740 0
1643838 1639290 0
1643838 1675774 0
1643838 1638391 0
1643838 1672997 0
1643838 1673241 0
1643838 1797645 0
1643838 1678070 0
1643838 1751218 0
1643838 1654376 0
1643838 1658601 0
1643838 1727188 0
1643838 1634832 0
1643838 1712661 0
1643838 16

1636198 1704316 0
1636198 1686751 0
1636198 1797887 0
1636198 1511443 0
1636198 1736271 0
1636198 1704733 0
1636198 1810160 0
1636198 1719167 0
1636198 1679612 0
1636198 1698008 0
1636198 1808800 0
1636198 1676553 0
1636198 1761793 0
1636198 1688783 0
1636198 1735651 0
1635896 1779251 0
1635891 1776376 0
1635891 1634832 0
1635273 1733916 0
1635273 1687095 0
1635273 1717885 0
1635273 1734418 0
1635273 1643646 0
1635273 1673183 0
1635274 1701584 0
1635274 1756056 0
1635274 1791315 0
1635274 1650169 0
1635274 1704354 0
1635274 1708997 0
1635274 1757128 0
1635274 1756050 0
1635274 1663480 0
1635274 1641739 0
1635274 1719184 0
1635274 1681014 0
1635274 1812606 0
1635274 1702662 0
1635274 1693674 0
1635274 1655949 0
1635274 1685564 0
1635274 1765188 0
1635274 1794148 0
1635274 1731081 0
1635274 1698923 0
1635274 1646264 0
1635274 1810946 0
1635274 1724939 0
1635274 1710019 0
1635274 1812340 0
1635274 1670933 0
1635274 1811918 0
1635274 1696343 0
1635274 1798035 0
1635274 1740109 0
1635274 17

1630825 1680231 0
1630825 1686352 0
1630825 1650012 0
1630825 1705680 0
1630825 1702289 0
1630825 1805357 0
1630825 1802001 0
1630825 1654943 0
1630825 1711312 0
1630825 1699383 0
1630825 1777681 0
1630825 1753421 0
1630825 1783480 0
1630825 1763227 0
1630825 1799857 0
1630825 1750517 0
1630825 1740015 0
1630825 1735948 0
1630825 1631146 0
1630825 1723687 0
1630825 1728661 0
1630825 1753095 0
1630825 1820157 0
1630825 1781329 0
1630825 1808771 0
1630825 1817353 0
1630825 1622500 0
1630825 1702183 0
1630825 1677374 0
1630825 1737810 0
1630825 1632745 0
1630825 1628958 0
1630825 1630830 0
1630825 1727272 0
1630825 1752411 0
1630825 1742685 0
1630825 1634597 0
1630825 1659150 0
1630825 1708616 0
1630825 1673043 0
1630825 1805311 0
1630825 1797900 0
1630825 1797508 0
1630825 1701473 0
1630825 1728391 0
1630825 1667218 0
1630825 1631312 0
1630825 1648393 0
1630825 1813878 0
1630825 1693690 0
1630825 1726359 0
1630825 1729548 0
1630825 1632420 0
1630825 1635505 0
1630825 1642687 0
1630825 16

...

1246787 1401026 0
1246787 1370430 0
1246787 1320087 0
1246787 1508867 0
1246787 1704460 0
1246787 1616005 0
1246787 1608623 0
1246787 1625356 0
1246787 1429441 0
1246787 1477825 0
1246787 1775646 0
1246787 1457602 0
1246787 1327727 0
1247559 1772396 0
1246784 1290738 0
1246784 1632762 0
1246784 1296485 0
1246784 1500198 0
1246784 1359288 0
1246784 1339767 0
1246784 1304943 0
1246784 1306497 0
1246784 1318341 0
1246784 1381399 0
1246784 1646006 0
1246784 1503617 0
1246784 1296613 0
1246784 1750906 0
1246784 1711109 0
1246784 1305000 0
1246784 1309585 0
1246784 1448297 0
1246784 1770283 0
1246784 1335419 0
1246783 1399054 0
1246783 1317671 0
1246783 1756371 0
1246783 1518142 0
1246783 1429598 0
1246783 1477591 0
1246785 1311410 0
1246785 1287757 0
1246785 1381431 0
1246785 1257742 0
1246785 1502922 0
1246785 1339916 0
1246785 1309585 0
1246785 1755851 0
1246785 1287944 0
1246367 1430761 0
1246367 1290736 0
1246367 1335714 0
1246367 1506404 0
1246367 1590893 0
1246367 1308735 0
1246367 15

1244669 1328977 0
1244669 1374224 0
1244669 1508617 0
1244669 1376135 0
1244669 1492729 0
1244669 1409533 0
1244669 1484856 0
1244669 1393728 0
1244669 1590031 0
1244669 1398054 0
1244669 1408517 0
1244669 1246012 0
1244669 1589448 0
1244669 1676410 0
1244669 1668145 0
1244669 1487550 0
1244669 1395619 0
1244669 1514047 0
1244669 1355347 0
1244669 1782531 0
1244669 1599853 0
1244669 1339796 0
1244669 1431997 0
1244669 1382177 0
1244669 1658601 0
1244669 1425866 0
1244669 1351718 0
1244669 1496440 0
1244669 1632255 0
1244669 1679601 0
1244669 1346408 0
1244669 1333404 0
1244669 1479455 0
1244669 1651244 0
1244669 1400838 0
1244669 1291137 0
1244669 1641769 0
1244669 1697490 0
1244669 1318838 0
1244669 1427029 0
1244669 1406194 0
1244669 1358225 0
1244669 1429612 0
1244669 1308916 0
1244669 1490894 0
1244669 1366203 0
1244669 1477464 0
1244669 1255316 0
1244669 1407474 0
1244669 1514034 0
1244669 1360296 0
1244669 1449094 0
1244669 1755856 0
1244669 1309124 0
1244669 1375491 0
1244669 13

1258337 1456158 0
1258337 1813328 0
1258337 1451747 0
1258337 1287944 0
1258337 1727597 0
1258337 1339723 0
1258337 1395229 0
1258337 1316578 0
1258337 1382013 0
1258337 1420006 0
1258337 1299843 0
1258337 1238810 0
1258337 1294778 0
1258337 1351705 0
1258337 1358394 0
1244131 1256109 0
1244131 1256587 0
1244131 1278588 0
1244131 1373340 0
1244131 1374496 0
1244131 1268357 0
1244131 1662482 0
1244131 1342820 0
1244131 1340334 0
1244128 1517486 0
1244128 1385189 0
1244128 1631008 0
1244128 1637672 0
1244128 1397544 0
1244128 1290776 0
1244128 1596766 0
1244128 1736536 0
1244128 1339848 0
1244128 1257865 0
1244128 1653168 0
1244128 1330325 0
1244128 1664082 0
1244128 1519424 0
1244128 1369123 0
1244128 1505176 0
1244128 1416656 0
1244128 1699566 0
1244128 1323476 0
1244128 1299990 0
1244128 1608880 0
1244128 1613333 0
1244128 1263695 0
1244128 1285966 0
1244128 1513309 0
1244128 1740329 0
1244128 1246144 0
1244128 1364504 0
1244128 1315449 0
1244128 1512930 0
1244128 1494347 0
1244128 15

1243865 1381354 0
1243865 1503351 0
1243865 1338179 0
1243865 1339787 0
1243865 1387752 0
1243865 1754978 0
1243865 1429447 0
1243865 1669345 0
1243865 1306915 0
1243865 1300425 0
1243865 1429595 0
1243865 1598286 0
1243865 1377255 0
1243865 1395973 0
1243865 1495990 0
1243865 1498308 0
1260959 1424622 0
1260959 1297223 0
1260959 1290703 0
1260959 1372859 0
1260959 1365308 0
1260959 1315950 0
1260959 1307847 0
1260959 1256608 0
1260959 1290651 0
1260959 1330046 0
1260959 1658532 0
1260959 1291207 0
1260959 1263394 0
1260959 1614085 0
1260959 1305306 0
1260959 1309878 0
1260959 1421703 0
1260959 1376285 0
1260959 1511301 0
1260959 1380078 0
1260959 1315335 0
1260959 1322917 0
1260959 1320562 0
1260959 1261919 0
1260959 1309113 0
1260959 1379972 0
1260959 1796484 0
1260959 1795420 0
1260959 1336075 0
1260959 1267680 0
1260959 1245031 0
1260959 1306771 0
1260959 1264502 0
1260959 1276513 0
1260959 1303909 0
1260959 1297494 0
1260959 1283180 0
1260959 1321008 0
1260959 1415459 0
1260959 12

1243425 1381343 0
1243425 1593765 0
1243425 1404710 0
1243425 1275619 0
1243425 1296393 0
1243425 1299000 0
1243425 1708929 0
1243425 1257623 0
1243425 1645660 0
1243425 1246913 0
1243425 1391805 0
1243425 1272715 0
1243425 1298763 0
1243425 1381186 0
1243425 1272844 0
1243425 1254055 0
1243425 1221253 0
1243425 1334520 0
1243425 1384255 0
1243425 1294382 0
1243425 1250395 0
1243425 1507481 0
1243425 1253131 0
1243425 1338098 0
1243425 1340478 0
1243425 1432767 0
1243425 1208850 0
1243425 1252732 0
1243425 1285488 0
1243425 1288887 0
1243425 1514260 0
1243425 1466218 0
1243425 1266786 0
1243425 1310275 0
1243425 1253554 0
1243425 1266255 0
1243425 1232361 0
1243425 1310401 0
1243425 1333249 0
1243425 1456809 0
1243425 1246161 0
1243425 1339846 0
1243425 1480374 0
1243425 1327249 0
1243425 1380460 0
1243425 1247063 0
1243425 1257742 0
1243425 1389194 0
1243425 1429440 0
1243425 1253388 0
1243425 1432707 0
1243425 1294106 0
1243425 1395773 0
1243425 1601384 0
1243425 1664329 0
1243425 13

In [ ]:
G.add_edges_from(edges)

In [ ]:
print('Nodes: {}\tEdges:{}'.format(G.number_of_nodes(), G.number_of_edges()))

In [ ]:
data_json = nx.node_link_data(G)

In [ ]:
df_nodes = pd.DataFrame(data_json['nodes'])
df_links = pd.DataFrame(data_json['links'])

In [ ]:
df_nodes.sample(10)

In [ ]:
df_links.sample(10)

In [ ]:
df_nodes.set_index('id').to_csv('nodes.csv',index=True)

In [ ]:
df_links.to_csv('links.csv',index=False)

In [ ]:
df_links.set_index('id').to_csv('links.csv',index=True)

### The most cited network

turn documents into pandas dataframe

In [ ]:
df = pd.DataFrame(documents)

drop rows with NaN

In [ ]:
df = df.dropna()

In [ ]:
df.sample(10)

create a new column with the number of citations

In [ ]:
df['num_citations' ] = df['citations_ids'].apply(lambda x: len(x))

and order by the number of citations

In [ ]:
df = df.sort_values(by='num_citations', ascending=False)

In [ ]:
df.head(20)

to create the network diagram set the minimum number of citations for a document to be kept into the analysis

In [ ]:
min_citations = 300

build a set with the ids of the docs that meet such requirement

In [ ]:
most_cited_ids = set(df[df.num_citations >= min_citations]['_id'])

and then create a new column with the list of each document citations that are in the most cited articles set

In [ ]:
df['most_cited'] = df.citations_ids.apply(
    lambda citations_list: [citation_id for citation_id in citations_list
                           if citation_id in most_cited_ids] 
)

In [ ]:
df.head()

and then create a new dataframe that only contains the documents that cite at least one of the most cited articles

In [ ]:
most_cited_df = df.loc[
    df.most_cited.apply(lambda most_cited: most_cited != [])
]

In [ ]:
most_cited_df = df.loc[
    df._id.apply(lambda _id: _id in most_cited_ids)
]

In [ ]:
most_cited_df.head(10)

order the dataframe by date

In [ ]:
most_cited_df.created = pd.to_datetime(most_cited_df.created)

In [ ]:
most_cited_df = most_cited_df.sort_values(by="created", ascending = False)

In [ ]:
most_cited_df = most_cited_df.reset_index(drop=True)

In [ ]:
most_cited_df.head(10)

In [ ]:
most_cited_df.loc[0]['metadata']['titles'][0]['title']

create a graph from the data in the dataframe

In [ ]:
G = nx.Graph()

build a list of nodes

In [ ]:
nodes = [
(int(most_cited_df.loc[i]['_id']),
  {'num_citations':int(most_cited_df.loc[i]['num_citations']),'title': most_cited_df.loc[i]['metadata']['titles'][0]['title']})
for i in range(0,most_cited_df.shape[0])]

In [ ]:
nodes[0:5]

add nodes to the network

In [ ]:
G.add_nodes_from(nodes)

add edges to the network

In [ ]:
for i in range(0,most_cited_df.shape[0]):
    article = most_cited_df.loc[i]
    edges = [(int(article._id),int(citation)) for citation in article.most_cited]
    G.add_edges_from(edges)

In [ ]:
len(G.nodes)

In [ ]:
len(G.edges)

In [ ]:
most_cited_df.shape

In [ ]:
data_json = nx.node_link_data(G)

In [ ]:
data_json

In [ ]:
import json

with open('data.json', 'w') as outfile:
    json.dump(data_json,outfile)

In [ ]:
data_json

In [ ]:
len(G.nodes)

In [ ]:
nx.write_gexf(G, "data.gexf")

In [ ]:
import matplotlib.pyplot as plt

# nx.draw(G, with_labels=False, font_weight='bold')
plt.subplots(figsize=[10,10])

# nx.draw_shell(G, nlist=[range(5, 10), range(5)], with_labels=True, font_weight='bold')

options = {
    'node_color': 'black',
    'node_size': 16,
    'width': 1,
}

nx.draw(G, **options)

In [ ]:
DG = nx.DiGraph()

In [ ]:
for i in range(0,most_cited_df.shape[0]):
    article = most_cited_df.loc[i]
    edges = [(citation,article._id) for citation in article.most_cited]
    DG.add_edges_from(edges)

In [ ]:
nx.write_gexf(G, "data.gexf")

In [ ]:
import matplotlib.pyplot as plt

# nx.draw(G, with_labels=False, font_weight='bold')
plt.subplots(figsize=[10,10])

# nx.draw_shell(G, nlist=[range(5, 10), range(5)], with_labels=True, font_weight='bold')

options = {
    'node_color': 'black',
    'node_size': 16,
    'width': 0.1,
}

nx.draw_circular(DG, **options)